In [89]:
from scipy.stats import norm
from time import time
import random    
import pandas as pd

In [90]:
class Cycliste:
    
    def log(self):
        try:
            print "logging",
            with open("./cyclistes.csv", "a") as f:
                df = pd.DataFrame(self.__dict__)
                f.write(df.to_csv())
                f.write("\n")
        except Exception as e:
            print e
            
        
    def __init__(self, identifiant):
        import time
        from scipy.stats import norm
        import random
        homme   = "homme"
        femme   = "femme"
        age_min = 15
        age_max = 80
        sexe    = random.choice([homme, femme])
        age     = random.randint(age_min ,age_max)
        sportif = random.choice([-0.5, 0, 2, 4, 6 ])
        if age < 35:
            facteur = norm( loc =1.2, scale=0.2 )
        elif age <50:
            facteur = norm( loc =1  , scale=0.2 )
        elif age <90:
            facteur = norm( loc =0.7, scale=0.2 )

        if sexe == homme:
            vitesse_de_base = 20
            nb_km = norm( loc =7, scale=2 )
        else:
            vitesse_de_base = 15
            nb_km = norm( loc =10, scale=2 )
        vitesse_moyenne = vitesse_de_base * facteur.rvs(1) + sportif
        
        self.sportif = sportif
        self.age = age
        self.sexe = sexe
        self.nb_km = nb_km.rvs(1)[0]
        self.vitesse = vitesse_moyenne[0]
        self.sur_velo =False
        self.debug = False
        self.nb_trajet = 0
        self.durees = []
        self.trajets = []
        self.velo = False
        self.identifiant = identifiant
        

    def prend_velo(self, station):
        if self.sur_velo:
            if self.debug : print "deja sur velo",
            pass
        else:
            import time
            velo = station.prendre_velo()
            if velo:
                velo.disponible      = False
                self.sur_velo        = True
                self.heure_de_depart = time.time()
                try:
                    nb_km_local          = norm( loc =self.nb_km, scale=self.nb_km/float(3) ).rvs(1)[0]
                except Exception as e:
                    print "pbm sur nb_km_local = self.nb_km = %s " %self.nb_km
                    print e
                    nb_km_local = 15
                try:
                    vitesse_local        = norm(loc = self.vitesse, scale = self.vitesse/float(10)).rvs(1)[0]
                    vitesse_local        = vitesse_local * velo.performance
                except:
                    print "pbm sur vitesse_local"
                    print "self.vitesse = %s"%self.vitesse
                    vitesse_local = self.vitesse
                self.nb_km_trajet    = nb_km_local
                duree                = nb_km_local / vitesse_local
                
                self.velo = velo
                
                self.heure_d_arrivee =  self.heure_de_depart + duree
                if self.debug : print "arrivee dans %s a %s"%(duree, self.heure_d_arrivee),
                if velo.performance <0.1:
                    duree = 0
                    self.durees.append( duree)
                    self.heure_d_arrivee =  self.heure_de_depart
                    self.nb_km_trajet    = 0
                    self.rendre_velo(station)
                elif velo.performance <0.3:
                    duree = duree / 10
                    self.durees.append( duree)
                    self.heure_d_arrivee =  self.heure_de_depart + duree
                    self.nb_km_trajet    = self.nb_km_trajet / float(10)
                    self.rendre_velo(station)
                elif velo.performance <0.5:
                    duree = duree / 5
                    self.durees.append( duree)
                    self.nb_km_trajet    = self.nb_km_trajet / float(5)
                    self.heure_d_arrivee =  self.heure_de_depart + duree 
                else:
                    self.durees.append( duree)
                    
            else:
                if self.debug : print "pas de velo ici",


    def rendre_velo(self, station):        
        import time
        if not self.sur_velo:
            if self.debug : print "pas en chemin",
            return 0
        if time.time() > self.heure_d_arrivee:
            if station.rendre_velo(self.velo, self.nb_km_trajet):
                #if self.debug : print "plus sur velo",
                duree_constatee = self.heure_d_arrivee - self.heure_de_depart
                self.trajets.append((self.heure_de_depart, self.heure_d_arrivee, self.nb_km_trajet, self.velo.nom ))
                self.sur_velo =False
                self.nb_trajet = self.nb_trajet + 1
                self.velo = False
            else:
                if self.debug : print "impossible de rendre sur station %s"%(station),
                pass
        else:
            if self.debug : print "pas encore arrive",
            pass
        






In [91]:
class Velo:
    def __init__(self, nom, station):
        from scipy.stats import norm
        import time 
        self.nom          = nom
        performance       = norm( loc =0.9, scale=0.2 )
        self.performance  = performance.rvs(1)[0]
        self.station      = [station]
        self.problemes    = []
        self.degradations = []
        self.performances = []
        self.debug        = False
        self.nb_km_trajet = []
        self.heures_rendu = []
        self.disponible   = True
    
    def rendu(self, station, nb_km_trajet):
        import time
        self.disponible = True
        if self.debug : print("velo %s rendu sur station %s"%(self, station))
            
        problemes       = [0] * 50
        problemes_bis   = {"pedale" : 0.2, "roue"  : 1 , "degonfle" : 0.3, "freins" : 0.05 , 0 : 0}
        problemes.extend(problemes_bis.keys())
        probleme        = random.choice(problemes)
        
        self.problemes.append(probleme)
        degradation = sum([ problemes_bis[x] for x in self.problemes if x in problemes_bis.keys()])
        
        self.performance = self.performance - self.performance*degradation
        self.performances.append(self.performance)
        self.degradations.append(degradation)
        self.nb_km_trajet.append(nb_km_trajet)
        self.station.append(station)
        self.heures_rendu.append(time.time())
        

    def __str__(self):
        return "%s"%self.nom

    def en_utilisation(self, station):
        self.disponible = False
        station.occupee = False


In [92]:
class Station:
    def __init__(self, nom ):
        import random
        import time
        self.nom = nom
        self.nb_plot  = random.choice([10,15,20, 30])
        self.nb_libre = random.randint(0, self.nb_plot )
        self.nb_velos = self.nb_plot - self.nb_libre
        self.velos    = []
        noms = get_names()
        for n in range(self.nb_velos ):
            nom_velo="velo_"+"".join(random.choice(noms))
            self.velos.append(Velo(nom_velo , self))
        
        print "created %s velos"%len(self.velos),
        self.debug = False
        self.avaries = []
        self.disponibilites = [("init", time.time(), self.nb_libre, self.nb_velos)]
        self.arrete = False

    def avarie_possible(self):
        import math
        import random
        import time
        avaries     = [0]*100000
        plot        = "plot mort"
        half        = "50% mort"
        network     = "reseau"
        electricite = "eteint"
        bug         = "bug"
        problemes   = { plot       : 20 ,
                       half        : 10 ,
                       network     :  5 ,
                       electricite : 30 , 
                       bug         :  3 }
        
        for k,v in problemes.iteritems():
            avaries.extend([k] * v)
            
        if plot in self.avaries : avaries.extend( [plot] * 60 )
        if half in self.avaries : avaries.extend( [half] *100 )
        
        avarie = random.choice(avaries)
        self.avaries.extend([avarie])
        if avarie == plot :
            self.nb_libre = self.nb_libre -1
        elif avarie == half:
            import math
            self.nb_libre = math.floor(self.nb_libre - (self.nb_plot / float(2)))
        elif avarie == network :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == electricite :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == bug :
            if self.nb_libre >0:
                self.nb_libre = self.nb_libre - random.randint(0, self.nb_libre)
        if self.nb_libre <0:
            self.nb_libre = 0            
        self.disponibilites.append(("%s"%avarie, time.time(), self.nb_libre, self.nb_velos))
        with open("./stations.csv", "a") as f:
            to_write = "%s, %s,%s,%s"%(self.nom, time.time(), self.nb_libre, self.nb_velos)
            f.write("%s\n"%to_write)
            
    def rendre_velo(self, velo, nb_km_trajet):
        import time
        self.avarie_possible()
        if self.nb_libre >0:
            self.velos.append(velo)
            velo.rendu(self, nb_km_trajet)
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre - 1
            self.disponibilites.append(("rendre", time.time(), self.nb_libre, self.nb_velos))
            return 1
        return 0

    def prendre_velo(self):
        """
        retourne un vélo ou 0 en cas d'erreur.
        """
        import time
        self.avarie_possible()
        if self.velos and not self.arrete :
            to_return     = self.velos.pop()
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre + 1
            self.disponibilites.append(("prendre",time.time(), self.nb_libre, self.nb_velos))
            return to_return

        return 0
    def __str__(self):
        return "station %s : self.nb_libre = %s, self.nb_velos =%s "%(self.nom,self.nb_libre, self.nb_velos)


In [121]:
def get_names(nb_lettres=5, n = 10000, prefix=""):
    """
    retourne n mot de nb_lettres avec prefix.
    """
    import itertools
    from random import shuffle
    lettres = "azertyuiopqsdfghjklmwxcvbn1234567890"
    names   = []
    for i in itertools.combinations(lettres, nb_lettres):
        name = "".join(i)
        name = prefix + name
        names.append(name)
        n = n -1
        if n <2:
            break
    shuffle(names)
    return names

In [117]:
names = get_names()
stations = [ Station("".join(random.choice(names))) for i in range(1000)]

created 1 velos created 6 velos created 20 velos created 12 velos created 1 velos created 27 velos created 24 velos created 5 velos created 2 velos created 1 velos created 12 velos created 6 velos created 12 velos created 14 velos created 12 velos created 9 velos created 10 velos created 7 velos created 7 velos created 1 velos created 6 velos created 5 velos created 11 velos created 9 velos created 21 velos created 11 velos created 5 velos created 23 velos created 16 velos created 14 velos created 19 velos created 10 velos created 22 velos created 9 velos created 0 velos created 21 velos created 3 velos created 5 velos created 11 velos created 0 velos created 13 velos created 10 velos created 0 velos created 2 velos created 16 velos created 0 velos created 26 velos created 10 velos created 6 velos created 15 velos created 20 velos created 4 velos created 9 velos created 2 velos created 22 velos created 0 velos created 7 velos created 14 velos created 11 velos created 17 velos created 2

In [122]:

nb_velos = sum([len(station.velos) for station in stations])
nb_velos

9280

In [123]:
names     = get_names(prefix="cycliste_", n=nb_velos+10)
cyclistes = [Cycliste(names[n]) for n in range(nb_velos)]

In [124]:
debug = False
from time import sleep
from time import time
n = 0  
while n <20 : 
    print n
    ancien_en_route = set([cycliste.identifiant for cycliste in cyclistes if     cycliste.sur_velo])
    ancien_a_pied   = set([cycliste.identifiant for cycliste in cyclistes if not cycliste.sur_velo])
    
    en_route = [cycliste for cycliste in cyclistes if     cycliste.sur_velo]
    a_pied   = [cycliste for cycliste in cyclistes if not cycliste.sur_velo]
    #print "%s - %s  "%(len(en_route) , len(a_pied))
    for cycliste in en_route:
        cycliste.rendre_velo(random.choice(stations))
    for cycliste in a_pied:
        cycliste.prend_velo(random.choice(stations))
    new_en_route = set([cycliste.identifiant for cycliste in cyclistes if     cycliste.sur_velo])
    new_a_pied   = set([cycliste.identifiant for cycliste in cyclistes if not cycliste.sur_velo])
    print "chgt = %s - %s  "%(len(new_en_route.difference(ancien_en_route)) , len(new_a_pied.difference(ancien_a_pied)))
    if new_en_route == ancien_en_route:
        print "pbm sur en route : ", 
    if new_a_pied == ancien_a_pied :
        print "pbm sur a pied : "
    
    n += 1

0
pbm sur nb_km_local = self.nb_km = -0.00582623838817 
Domain error in arguments.
chgt = 6202 - 0  
1
chgt = 2760 - 6081  
2
pbm sur nb_km_local = self.nb_km = -0.00582623838817 
Domain error in arguments.
chgt = 4728 - 2758  
3
chgt = 3557 - 4658  
4
pbm sur nb_km_local = self.nb_km = -0.00582623838817 
Domain error in arguments.
chgt = 3977 - 3536  
5
chgt = 3707 - 3899  
6
chgt = 3733 - 3721  
7
pbm sur nb_km_local = self.nb_km = -0.00582623838817 
Domain error in arguments.
chgt = 3477 - 3697  
8
chgt = 3360 - 3466  
9
pbm sur nb_km_local = self.nb_km = -0.00582623838817 
Domain error in arguments.
chgt = 3182 - 3328  
10
chgt = 3042 - 3161  
11
pbm sur nb_km_local = self.nb_km = -0.00582623838817 
Domain error in arguments.
chgt = 2909 - 3082  
12
chgt = 2678 - 2865  
13
pbm sur nb_km_local = self.nb_km = -0.00582623838817 
Domain error in arguments.
chgt = 2499 - 2671  
14
chgt = 2263 - 2509  
15
chgt = 2076 - 2236  
16
pbm sur nb_km_local = self.nb_km = -0.00582623838817 
Domai

In [125]:
nb_trajet = sum([cycliste.nb_trajet for cycliste in cyclistes])
nb_trajet

98144

In [126]:
import pandas as pd
tous_velos = []
velos_en_station  = [ velo for station_ in stations for velo in station_.velos  if station_.velos ]
velos_en_partance = [ cycliste_.velo for cycliste_ in cyclistes   if cycliste_.velo ]
tous_velos.extend(velos_en_station)
tous_velos.extend(velos_en_partance)
tous_velos = pd.DataFrame([x.__dict__ for x in tous_velos])
tous_velos

,debug,degradations,disponible,heures_rendu,nb_km_trajet,nom,performance,performances,problemes,station
0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.3, 0.3, 0.3, 0.6...",True,"[1485039591.44, 1485039639.14, 1485039655.25, ...","[3.39662687515, 10.0625693979, 8.71315016249, ...",velo_ej8,5.385403e-02,"[0.981305221785, 0.981305221785, 0.98130522178...","[0, 0, 0, 0, 0, 0, 0, 0, 0, degonfle, 0, 0, de...","[station tk3 : self.nb_libre = 11, self.nb_vel..."
1,False,"[0, 0, 0, 0, 0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, ...",True,"[1485039593.47, 1485039637.09, 1485039655.44, ...","[4.91484849246, 3.26069760281, 7.31407613122, ...",velo_ql3,0.000000e+00,"[0.835112041275, 0.835112041275, 0.83511204127...","[0, 0, 0, 0, 0, degonfle, 0, 0, 0, 0, 0, 0, 0,...","[station gwx : self.nb_libre = 4, self.nb_velo..."
2,False,[],True,[],[],velo_iq9,7.220789e-01,[],[],"[station g25 : self.nb_libre = 7, self.nb_velo..."
3,False,[0],True,[1485039591.75],[3.30314122908],velo_v68,9.674481e-01,[0.967448063692],[0],"[station f90 : self.nb_libre = 16, self.nb_vel..."
4,False,[0],True,[1485039592.43],[16.6070866516],velo_dgv,8.081777e-01,[0.808177675052],[0],"[station qhk : self.nb_libre = 15, self.nb_vel..."
5,False,[0],True,[1485039593.26],[10.4039396133],velo_k10,1.007453e+00,[1.00745276703],[0],"[station fg9 : self.nb_libre = 9, self.nb_velo..."
6,False,[0],True,[1485039594.67],[7.84645601636],velo_jn8,1.193282e+00,[1.19328223822],[0],"[station ts7 : self.nb_libre = 0, self.nb_velo..."
7,False,"[1, 1, 1, 1, 1, 1, 1.05, 1.05, 1.05, 1.05, 1.0...",True,"[1485039595.13, 1485039624.78, 1485039628.17, ...","[7.85041059592, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",velo_lb9,0.000000e+00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[roue, 0, 0, 0, 0, 0, freins, 0, 0, 0, 0, 0, 0...","[station m49 : self.nb_libre = 10, self.nb_vel..."
8,False,"[0, 0, 0, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, ...",True,"[1485039594.19, 1485039610.72, 1485039638.59, ...","[8.29991716002, 17.7029470945, 10.5515426543, ...",velo_pjm,5.468915e-01,"[1.06535496809, 1.06535496809, 1.06535496809, ...","[0, 0, 0, freins, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[station rxn : self.nb_libre = 9, self.nb_velo..."
9,False,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, ...",True,"[1485039591.53, 1485039740.18, 1485039762.51, ...","[9.56703008868, 13.4980293783, 12.7565222511, ...",velo_ojn,0.000000e+00,"[0.840622187692, 0.840622187692, 0.84062218769...","[0, 0, 0, 0, 0, 0, 0, 0, roue, 0, 0, 0, 0, 0, ...","[station h19 : self.nb_libre = 2, self.nb_velo..."


In [132]:
tous_velos.to_csv("velos.csv")

In [127]:
stations_df = pd.DataFrame([x.__dict__ for x in stations])

In [131]:
stations_df.to_csv("stations.csv")

In [128]:
a = pd.DataFrame([x.__dict__ for x in cyclistes])

In [130]:
a.to_csv("cyclistes.csv")

In [133]:
from sklearn import ensemble

In [138]:
import numpy as np

In [143]:

a["duree_moyenne"] = a.durees.apply(np.mean)
a["duree_max"] = a.durees.apply(np.max)
a["duree_min"] = a.durees.apply(np.min)
a["duree_std"] = a.durees.apply(np.std)

In [147]:
a.dtypes

age                  int64
debug                 bool
durees              object
heure_d_arrivee    float64
heure_de_depart    float64
identifiant         object
nb_km              float64
nb_km_trajet       float64
nb_trajet            int64
sexe                object
sportif            float64
sur_velo              bool
trajets             object
velo                object
vitesse            float64
duree_moyenne      float64
duree_max          float64
duree_std          float64
duree_min          float64
dtype: object

In [186]:
sub_cyclistes = a.select_dtypes(include=["int64", "float64", ])

In [201]:
import sklearn

In [202]:
sklearn.__version__

'0.17.1'

In [249]:
!head cyclistes.csv 

,age,debug,durees,heure_d_arrivee,heure_de_depart,identifiant,nb_km,nb_km_trajet,nb_trajet,sexe,sportif,sur_velo,trajets,velo,vitesse
0,74,False,"[1.2007435577726979, 1.7034748169690759, 1.169642696457895, 1.1306156426787952, 0.83090474232606815, 1.0372658200604634, 1.2947813660573495, 0.47798567176731155, 1.4399820101785892, 0.79199145004133442]",1485039992.32,1485039991.53,cycliste_aeti2,14.5322884419,9.36527406722,10,femme,-0.5,False,"[(1485039567.727287, 1485039568.9280307, 19.949912941655867, 'velo_ard'), (1485039611.364244, 1485039613.0677187, 14.97600232796098, 'velo_ol4'), (1485039657.825431, 1485039658.9950738, 17.611306526391136, 'velo_mw8'), (1485039698.110704, 1485039699.2413197, 17.823825932487921, 'velo_fl6'), (1485039740.877657, 1485039741.7085617, 11.476414077224709, 'velo_tn3'), (1485039784.135992, 1485039785.1732578, 13.353190801046702, 'velo_tjw'), (1485039830.500484, 1485039831.7952654, 14.740780323542078, 'velo_ap2'), (1485039882.330017, 1485039882.8080027, 6.67427

In [203]:
from sklearn import cross_validation 

In [204]:
sub_cyclistes['sexe'] = a['sexe']

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [205]:
sub_cyclistes['sexe'] = sub_cyclistes['sexe'].apply(lambda x:  1 if x=='femme' else -1)

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [206]:
Y = sub_cyclistes.sexe

In [207]:
X = sub_cyclistes.drop(["sexe"], axis=1)

In [221]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(X, Y)

In [239]:

xgboost = ensemble.GradientBoostingClassifier(max_depth =3, warm_start=True)

In [240]:
xgb_model = xgboost.fit(x_train, y_train, )

In [241]:
xgb_model.score(x_test, y_test)

0.84568965517241379

In [225]:
rf =ensemble.RandomForestClassifier(n_estimators=1000,n_jobs=-1, )
model = rf.fit(x_train, y_train,)

In [226]:
model.score(x_test, y_test)

0.8318965517241379

In [214]:
zip(model.feature_importances_, x_train.columns)

[(0.085007069228388854, 'age'),
 (0.00035595788979714277, 'heure_d_arrivee'),
 (0.00034394407590472771, 'heure_de_depart'),
 (0.27833933762766672, 'nb_km'),
 (0.038068444224202393, 'nb_km_trajet'),
 (0.028572347666705966, 'nb_trajet'),
 (0.039002236514533134, 'sportif'),
 (0.12073599069063468, 'vitesse'),
 (0.10520203617447933, 'duree_moyenne'),
 (0.14158067691994092, 'duree_max'),
 (0.15369889541058041, 'duree_std'),
 (0.0090930635771653157, 'duree_min')]

In [242]:
!git status

On branch master
Your branch is up-to-date with 'velib/master'.
Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   velib.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   velib.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.DS_Store
	.ipynb_checkpoints/
	Untitled.ipynb
	atlassian/
	cyclistes.csv
	cyclistes.json
	derby.log
	esilv/
	ibm_intro_python_et_spark/
	last_n
	linkedin/
	locanacelle.ipynb
	metastore_db/
	new_csfnews.csv
	profile_default/
	python/
	python_pour_python_2/
	python_pour_python_3/
	sauvegarde.csv
	sauvegarde_python_avant_2to3.zip
	seloger/
	stations.csv
	tensorflow/
	test
	test git.ipynb
	velos.csv

